# Modeling Runner's Times in Cherry Blossom Race

Daniel Byrne, Joanna Duran

9/19/19

## Abstract

Running for pleasure in a slightly competitive weekend run, often for charity has become a weekend pleaure for many a heath enthusiist across the United States. In this paper we examine the realationship between age of the participants and the race year to see if annual races have attracted the same or varying age participants as the years have gone on.  

## Introduction

Running for pleasure in a slightly competitive weekend run, often for charity has become a weekend pleaure for many a heath enthusiist across the United States. Races tend to be sponsored by corporate benefactors and the sucessful ones continue their the compietitions  yearly. Often also the race organizers post the race data on their website for anyone to consume. We study the realationship between age of the participants and the race year to see if races havae attracted the same or varying age participants as the years have gone on.  

In this paper we examine the data from the Cherry Blossom Ten Mile Run held in Washington D.C. held annually in early April when the cherry trees in bloom. The Cherry Blossom race started in 1973.  It is used as a lead in competition for elite runners planning to compete in the Boston Marathon. 

Earlier [research][1] by Kaplan and Nolan had shown that 1999 runners were typically older than the 2012 runners. We compare the age distributions of male runners across the years 1999 - 2014.  We will answer "How do the distributions change over the years?" and "Was it a gradual change?"

## Methods

We began by first investigating the data mining approach used by Nolan and Kaplan.  It was extensive and comprehensive approach that worked well for the race data as it was presented on the [Cherry Blossom Race Results website][2] at the time of the study.  Since its publicatin however, the  website has changed their data repository, and so we took this opportunity to explore new ways to mine the data that could possibily be extended across all years to include the original study range and the years competed since and also prior to 1999.  

The mining approach we took was to use the CRAN r package rvest to identify the table on the results page that is updated to contain the data specified by the search parameters and the page number.  We then looped through the pages of data over the the period of interest by varying the search parameters and recording the data in UTF-8 csv files for each year 1999-2014. 

The code to scrape the data in its newest form was simplier to implement and thus less error. prone than the original.  The code consisted of simmply 2 r function
- `getResults(uri, year = 1999)` - Loops over every available page of datafor the selected year.
- `getAll()` - Loops over the years 1999-2014 and saves each year's data in a csv file.

The scraping code r code is listed in `cbreaddata.r`

In [ ]:
# Loops over every page with data for the selected year
getResults <- function(uri, year = 1999) 

### Results
The results section will include intelligent discussion of the output of the work.  Please stick to the data and avoid commentary on the software including issues with errors, bugs or code.  Figures, tables, and diagrams are especially useful in this section.  Be sure to caption figures and describe them in a manner consistent with a professional report.  For example, if the assignment was to try a new method, compare the results with a previous standard method.  If multiple methods were used, compare the results to each other.  Describe what the data implies at a level of an educated reader in the field.

Refer to figures as 'Fig X.' such as Fig 1.  Figures can be presented inline or after the main text, but should always be a part of the main document.  It is asked that figures be generated by the software and not screen captures.  If you are uncertain how to generate a figure from a piece of software, feel free to ask.  Please note that all of the software packages utilized in this course are capable of producing graphics for export.  Label all axes, use units and other best practices.  Make the title descriptive of the data presented.  It is also acceptable to include your code inline for Jupyter, including graphics generation.

In [3]:
#Comments are not extra.  They are encouraged
%matplotlib inline
import numpy as np
#going to make a random function
x=np.linspace(-10,10)
y=x+1/10*(x**3)+20*np.sin(2*x)
txt='Figure 1: Do not forget to include a caption'
import matplotlib.pyplot as plt
fig = plt.figure()
ax1 = fig.add_axes((0.1, 0.2, 0.8, 0.7))
ax1.set_title("This is my title")
ax1.set_xlabel('X-axis')
ax1.set_ylabel('Y-axis')
# make the edge colors match the facecolors
ax1.scatter(x,y, c='r', edgecolors='face')
# center text
fig.text(.5, .05, txt, ha='center')
# resize the figure to match the aspect ratio of the Axes    
fig.set_size_inches(7, 8, forward=True)
plt.show()

ERROR: Error in parse(text = x, srcfile = src): <text>:2:1: unexpected input
1: #Comments are not extra.  They are encouraged
2: %matplotlib inline
   ^


### Conclusion
Summarize your results.  Again, do so in a professional manner.  Conclude with what was learned and any possible follow ups.  Avoiding again commenting on errors and software issues.  It is appropriate to talk about methods and how the results compare.

## References

- [1]: http://rdatasciencecases.org/ "Data Science in R. A Case Studies Approach to Computational Reasoning and Problem Solving", Deborah Nolan, Duncan Temple Lang, chapter 1 Predicting Location via Indoor Positioning Systems
- [2]: http://www.cballtimeresults.org/performances?utf8=%E2%9C%93&section=10M&year=2019&division=Overall+Men Credit Union Cheery Blossom Ten Mile Run Race Results